In [54]:
# from tqdm.notebook import trange, tqdm
from pathlib import Path
from utilities import files
import os.path as op
import numpy as np
from os import sep, remove
import itertools as it
import json
from mne import read_epochs
import matlab.engine

In [46]:
parasite = matlab.engine.start_matlab()

In [71]:
def many_is_in(multiple, target):
    check_ = []
    for i in multiple:
        check_.append(i  in target)
    return any(check_)

def all_is_in(multiple, target):
    check_ = []
    for i in multiple:
        check_.append(i  in target)
    return all(check_)

def cat(options, target):
    for i in options:
        if i in target:
            return i

In [15]:
exclude=[
    "sub-002-ses-01",
    "sub-002-ses-02",
    "sub-006-ses-01",
    "sub-006-ses-02",
    "sub-008-ses-01"
]
ds_path = "/home/common/bonaiuto/cued_action_meg"

In [17]:
def scrape_epo_files(ds_path, exclude):
    subs_path = op.join(ds_path, "derivatives", "processed")
    sub_paths = files.get_folders_files(subs_path)[0]
    sub_paths.sort()
    subs = [i.split(sep)[-1] for i in sub_paths]
    epochs = {sub: {"motor": [], "visual1": [], "visual2": []} for sub in subs}
    epochs_all = []
    for sub_path in sub_paths:
        sub = sub_path.split(sep)[-1]
        sessions_paths = files.get_folders_files(sub_path)[0]
        sessions_paths = [i for i in sessions_paths if "ses-" in i]
        session_motor = {i.split(sep)[-1]: files.get_files(i, "autoreject-sub", "-motor-epo.fif")[2] for i in sessions_paths}
        session_visual1 = {i.split(sep)[-1]: files.get_files(i, "autoreject-sub", "-visual1-epo.fif")[2] for i in sessions_paths}
        session_visual2 = {i.split(sep)[-1]: files.get_files(i, "autoreject-sub", "-visual2-epo.fif")[2] for i in sessions_paths}
        session_motor = list(it.chain(*[session_motor[i] for i in session_motor.keys()]))
        session_visual1 = list(it.chain(*[session_visual1[i] for i in session_visual1.keys()]))
        session_visual2 = list(it.chain(*[session_visual2[i] for i in session_visual2.keys()]))
        session_motor = [i for i in session_motor if not many_is_in(exclude, i)]
        session_motor.sort()
        session_visual1 = [i for i in session_visual1 if not many_is_in(exclude, i)]
        session_visual2 = [i for i in session_visual2 if not many_is_in(exclude, i)]
        session_visual1.sort()
        session_visual2.sort()
        epochs[sub]["visual1"] = session_visual1
        epochs[sub]["visual2"] = session_visual2
        epochs[sub]["motor"] = session_motor
        epochs_all.extend(session_visual1 + session_visual2 + session_motor)
    return epochs, epochs_all

epochs, epochs_all = scrape_select_files(ds_path, exclude)

In [38]:
def get_res4(ds_path, sub, ses, run):
    ref_path = op.join(ds_path, "raw", sub, ses, "meg")
    blocks = files.get_folders_files(ref_path)[0]
    block = [i for i in blocks if "block-{}".format(run[1:]) in i][0]
    res4_path = files.get_files(block, "", ".res4")[2][0]
    return res4_path

In [53]:
def average_filter_convert(file_path, parasite, filt=False, l_freq=None, h_freq=None):
    path_split = file_path.split(sep)
    filename_core = path_split[-1].split(".")[0]
    sub = filename_core[11:18]
    ses = filename_core[19:25]
    run = filename_core[26:29]
    res4_path = get_res4(ds_path, sub, ses, run)
    dir_path = str(sep).join(path_split[:-1] + ["inverse", ""])
    files.make_folder(dir_path)
    
    filt_status = "_nofilt"
    if filt:
        filt_status = "_filt"
    
    output_file = "spm-converted{}_{}".format(filt_status, filename_core)
    output_path = op.join(dir_path, output_file)
    average_file = output_path + "-ave.fif"
    mat_output = output_path + ".mat"
    if not op.exists(mat_output):
        if not op.exists(average_file):
            epochs = read_epochs(file_path, verbose=False)
            epochs = epochs.average()
            if filt:
                epochs.filter(l_freq=l_freq, h_freq=h_freq)
            epochs.save(average_file)

        parasite.convert_mne_to_spm(res4_path, average_file, mat_output, 0, nargout=0)
        if op.isfile(average_file):
            remove(average_file)
        else:
            print(average_file.split(sep)[-1], "does not exists")

    else:
        print(mat_output.split(sep)[-1], "exists")

average_filter_convert(epochs_all[0], parasite)
average_filter_convert(epochs_all[0], parasite, filt=True, l_freq=None, h_freq=30)

In [114]:
subs_path = op.join(ds_path, "derivatives", "processed")
def get_files(target_path, suffix, strings=[""], prefix=None):
    paths = [str(path) for path in Path(target_path).rglob(suffix) if all_is_in(strings, str(path.name))]
    paths.sort()
    if isinstance(prefix, str):
        paths = [path for path in paths if Path(path).name.startswith(prefix)] 
    return paths

mat_paths = get_files(subs_path, "*.mat", strings=["filt"], prefix="spm-converted")
print(mat_paths)

['/home/common/bonaiuto/cued_action_meg/derivatives/processed/sub-001/ses-01/inverse/spm-converted_filt_autoreject-sub-001-ses-01-001-visual1-epo.mat', '/home/common/bonaiuto/cued_action_meg/derivatives/processed/sub-001/ses-01/inverse/spm-converted_nofilt_autoreject-sub-001-ses-01-001-visual1-epo.mat']


In [84]:
mat_paths

['/home/common/bonaiuto/cued_action_meg/derivatives/processed/sub-001/ses-01/inverse/spm-converted_filt_autoreject-sub-001-ses-01-001-visual1-epo.mat',
 '/home/common/bonaiuto/cued_action_meg/derivatives/processed/sub-001/ses-01/inverse/spm-converted_nofilt_autoreject-sub-001-ses-01-001-visual1-epo.mat']

In [119]:
def invert_multisurface(ds_path, mat_file_path, parasite, an_name="multilayer-11", layers=11):
    input_path = Path(mat_file_path)
    # input_path.stem, input_path.parent, input_path.name
    # op.join(input_path.parent, "res_" + input_path.stem + ".tsv")
    bits = input_path.name.split("_")[-1].split("-")
    sub = "-".join([bits[1], bits[2]])
    ses = "-".join([bits[3], bits[4]])
    run = bits[5]
    epo = bits[6]
    output_path = str(input_path.parent)
    mu_file = op.join(input_path.parent, "{}_MU_".format(an_name) + input_path.stem + ".tsv")
    it_file = op.join(input_path.parent, "{}_IT_".format(an_name) + input_path.stem + ".tsv")
    res_file = op.join(input_path.parent, "{}_res_".format(an_name) + input_path.stem + ".tsv")
    json_out_file = op.join(input_path.parent, "{}_invert-res_".format(an_name) + input_path.stem + ".json")
    
    parasite.invert_multisurface(
        str(ds_path), output_path, sub, ses, run, mat_file_path, 
        mu_file, it_file, res_file, json_out_file, 
        an_name, layers, epo, nargout=0
    )
    return sub, ses, run, bits, mu_file, it_file, mu_file, json_out_file

print(invert_multisurface(ds_path, mat_paths[0], parasite))

MatlabExecutionError: 
  File , line 0, in MATLABbatch system
Job execution failed. The full log of this run can be found in MATLAB command window, starting with the lines (look for the line showing the exact #job as displayed in this error message)
------------------ 
Running job #1
------------------

